# Data Wrangling

## Load the required dependencies


In [1]:
# Import frameworks
import pandas as pd

### Store the data as a local variable
The data frame is a Pandas object that structures your tabular data into an appropriate format. It loads the complete data in memory so it is now ready for preprocessing.

In [2]:
data_frame = pd.read_csv("1.1.4.Student_Scores_Data.csv")


### Dealing with null values
Null values during data analysis can cause runtime errors and unexpected results. It is important to identify null values and deal with them appropriately before training a model.

The isnull().sum() method call returns the null values in any column.

In [3]:
data_frame.isnull().sum()

Hours_Studied                  0
Attendance                     0
Parental_Involvement           0
Access_to_Resources            0
Extracurricular_Activities     0
Sleep_Hours                    0
Previous_Scores                0
Motivation_Level               0
Internet_Access                0
Tutoring_Sessions              0
Family_Income                  0
Teacher_Quality               78
School_Type                    0
Peer_Influence                 0
Physical_Activity              0
Learning_Disabilities          0
Parental_Education_Level      90
Distance_from_Home            67
Gender                         0
Exam_Score                     0
dtype: int64

To deal with the null variables from the dataset you can either:
 - Remove a row with dropna() method call
 - Replace missing values with fillna() method call and use mean value for numerical columns because it causes minimal changes in mathematical analysis while maintaining original data size

In [4]:
# Remove Null values for teacher quality
data_frame = data_frame.dropna(subset=['Teacher_Quality'])
data_frame.isnull().sum()

Hours_Studied                  0
Attendance                     0
Parental_Involvement           0
Access_to_Resources            0
Extracurricular_Activities     0
Sleep_Hours                    0
Previous_Scores                0
Motivation_Level               0
Internet_Access                0
Tutoring_Sessions              0
Family_Income                  0
Teacher_Quality                0
School_Type                    0
Peer_Influence                 0
Physical_Activity              0
Learning_Disabilities          0
Parental_Education_Level      86
Distance_from_Home            65
Gender                         0
Exam_Score                     0
dtype: int64

In [5]:
# Remove Null values for parental education level
data_frame = data_frame.dropna(subset=['Parental_Education_Level'])
data_frame.isnull().sum()

Hours_Studied                  0
Attendance                     0
Parental_Involvement           0
Access_to_Resources            0
Extracurricular_Activities     0
Sleep_Hours                    0
Previous_Scores                0
Motivation_Level               0
Internet_Access                0
Tutoring_Sessions              0
Family_Income                  0
Teacher_Quality                0
School_Type                    0
Peer_Influence                 0
Physical_Activity              0
Learning_Disabilities          0
Parental_Education_Level       0
Distance_from_Home            65
Gender                         0
Exam_Score                     0
dtype: int64

In [6]:
# Remove Null values for distance from home
data_frame = data_frame.dropna(subset=['Distance_from_Home'])
data_frame.isnull().sum()

Hours_Studied                 0
Attendance                    0
Parental_Involvement          0
Access_to_Resources           0
Extracurricular_Activities    0
Sleep_Hours                   0
Previous_Scores               0
Motivation_Level              0
Internet_Access               0
Tutoring_Sessions             0
Family_Income                 0
Teacher_Quality               0
School_Type                   0
Peer_Influence                0
Physical_Activity             0
Learning_Disabilities         0
Parental_Education_Level      0
Distance_from_Home            0
Gender                        0
Exam_Score                    0
dtype: int64

Null values were removed rather than replacing missing values as the features with null values were categorical data not numerical meaning finding the mean wouldn't work

### Remove Duplicates
Duplicate data can have detrimental effects on the machine learning model and outcomes, such as reducing data diversity and representativeness, which can lead to overfitting or a biased model.

The duplicated().sum() method call returns the count of duplicate rows in the data set.

In [7]:
data_frame.duplicated().sum()

np.int64(0)

The drop_duplicates() method call can be then stored back onto the data_frame variable removing the duplicates.

In [8]:
data_frame = data_frame.drop_duplicates()
data_frame.duplicated().sum()

np.int64(0)

### Replace data
I have decided not to replace any data and modify any values. This is for simplicity as changing something like Gender to be lower case rather than uppercase would mean changing the other categorical columns which is time consuming and inneffective.

### Remove outliers
Outliers may skew the analysis on numerical columns so it is important to address and potentially remove them. After using the first and third quartile on numerical data outliers were identified but extreme values can represent patterns for the model to address and use to generalize across different types of movies.

In [ ]:
#get the inter-quartile range on the Exam_Score column
print(data_frame['Exam_Score'].describe())
Q1 = data_frame['Exam_Score'].quantile(0.10)
Q3 = data_frame['Exam_Score'].quantile(0.90)
IQR = Q3 - Q1
print(f'Outliers are a Exam_Score above {Q3 + IQR * 1.5} or below {Q1 - IQR * 1.5}')

count    6378.000000
mean       67.252117
std         3.914217
min        55.000000
25%        65.000000
50%        67.000000
75%        69.000000
max       101.000000
Name: Exam_Score, dtype: float64
Outliers are a Exam_Score above 85.5 or below 49.5


The Exam Score column is the target for our model so filtering through outliers is not effective. Though there is a score of 101 which is in theory not possible and has the possibility to skew data so it will be eliminated.

In [16]:
# Remove single outlying value
data_frame = data_frame[data_frame['Exam_Score'] <= 100]
print(data_frame['Exam_Score'].describe())

count    6376.000000
mean       67.241688
std         3.870260
min        55.000000
25%        65.000000
50%        67.000000
75%        69.000000
max        99.000000
Name: Exam_Score, dtype: float64


### Scaling features to a common range
Scaling the features makes it easier for machine learning algorithms to find the optimal solution, as the different scales of the features do not influence them. This table shows the IQR range for all numerical columns which can then be used to scale the features, making it easier to identify the ranges to scale within.

In [18]:
import pandas as pd
import numpy as np

# Load the Dataset
data_frame = pd.read_csv('1.1.4.Student_Scores_Data.csv')

# Describe the dataset to get the required statistics
description = data_frame.describe().T

# Select only the required columns
description = description[['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]

# Display the description table
print("\nDescriptive statistics for each numerical column:")
print(description)


Descriptive statistics for each numerical column:
                    count       mean        std   min   25%   50%   75%    max
Hours_Studied      6607.0  19.975329   5.990594   1.0  16.0  20.0  24.0   44.0
Attendance         6607.0  79.977448  11.547475  60.0  70.0  80.0  90.0  100.0
Sleep_Hours        6607.0   7.029060   1.468120   4.0   6.0   7.0   8.0   10.0
Previous_Scores    6607.0  75.070531  14.399784  50.0  63.0  75.0  88.0  100.0
Tutoring_Sessions  6607.0   1.493719   1.230570   0.0   1.0   1.0   2.0    8.0
Physical_Activity  6607.0   2.967610   1.031231   0.0   2.0   3.0   4.0    6.0
Exam_Score         6607.0  67.235659   3.890456  55.0  65.0  67.0  69.0  101.0


In [19]:
scale_feature = 'Hours_Studied'

# Get the minimum and maximum values for the 'Hours_Studied' column
MIN_BP = data_frame[scale_feature].min()
MAX_BP = data_frame[scale_feature].max()

# Apply min-max scaling
data_frame[scale_feature] = (data_frame[scale_feature] - MIN_BP) / (MAX_BP - MIN_BP)

In [20]:
scale_feature = 'Attendance'

# Get the minimum and maximum values for the 'Attendance' column
MIN_BP = data_frame[scale_feature].min()
MAX_BP = data_frame[scale_feature].max()

# Apply min-max scaling
data_frame[scale_feature] = (data_frame[scale_feature] - MIN_BP) / (MAX_BP - MIN_BP)

In [21]:
scale_feature = 'Sleep_Hours'

# Get the minimum and maximum values for the 'Sleep_Hours' column
MIN_BP = data_frame[scale_feature].min()
MAX_BP = data_frame[scale_feature].max()

# Apply min-max scaling
data_frame[scale_feature] = (data_frame[scale_feature] - MIN_BP) / (MAX_BP - MIN_BP)

In [22]:
scale_feature = 'Previous_Scores'

# Get the minimum and maximum values for the 'Previous_Scores' column
MIN_BP = data_frame[scale_feature].min()
MAX_BP = data_frame[scale_feature].max()

# Apply min-max scaling
data_frame[scale_feature] = (data_frame[scale_feature] - MIN_BP) / (MAX_BP - MIN_BP)

In [23]:
scale_feature = 'Tutoring_Sessions'

# Get the minimum and maximum values for the 'Tutoring_Sessions' column
MIN_BP = data_frame[scale_feature].min()
MAX_BP = data_frame[scale_feature].max()

# Apply min-max scaling
data_frame[scale_feature] = (data_frame[scale_feature] - MIN_BP) / (MAX_BP - MIN_BP)

In [24]:
scale_feature = 'Physical_Activity'

# Get the minimum and maximum values for the 'Physical_Activity' column
MIN_BP = data_frame[scale_feature].min()
MAX_BP = data_frame[scale_feature].max()

# Apply min-max scaling
data_frame[scale_feature] = (data_frame[scale_feature] - MIN_BP) / (MAX_BP - MIN_BP)

In [25]:
scale_feature = 'Exam_Score'

# Get the minimum and maximum values for the 'Exam_Score' column
MIN_BP = data_frame[scale_feature].min()
MAX_BP = data_frame[scale_feature].max()

# Apply min-max scaling
data_frame[scale_feature] = (data_frame[scale_feature] - MIN_BP) / (MAX_BP - MIN_BP)

In [26]:
# Verify the scaled data
data_frame.describe()

,Hours_Studied,Attendance,Sleep_Hours,Previous_Scores,Tutoring_Sessions,Physical_Activity,Exam_Score
count,6607.000000,6607.000000,6607.000000,6607.000000,6607.000000,6607.000000,6607.000000
mean,0.441287,0.499436,0.504843,0.501411,0.186715,0.494602,0.265993
std,0.139316,0.288687,0.244687,0.287996,0.153821,0.171872,0.084575
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.348837,0.250000,0.333333,0.260000,0.125000,0.333333,0.217391
50%,0.441860,0.500000,0.500000,0.500000,0.125000,0.500000,0.260870
75%,0.534884,0.750000,0.666667,0.760000,0.250000,0.666667,0.304348
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


[!important] You need to save the calculations for each dataset you scale for scaling new values for prediction. Use 1.1.2.data.records.md to record this information.

## Save the wrangled data to CSV


In [28]:
data_frame.to_csv('../1.2.Feature_Engineering/wrangled_data.csv', index=False)